In [13]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import time
import datetime

In [15]:
X = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/X.csv')
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')
Y = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Y.csv')

In [16]:
cols_drop = ['TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END','TST_STA','XTST_END']
X.drop(cols_drop,axis=1,inplace=True)
Xtest.drop(cols_drop,axis=1,inplace=True)

In [20]:
for a in X.columns:
    scaler = StandardScaler()
    X[a] = scaler.fit_transform(X[a])
for a in Xtest.columns:
    scaler = StandardScaler()
    Xtest[a] = scaler.fit_transform(Xtest[a])

f:\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
f:\python35\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
f:\python35\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains

In [21]:
def abs_MAPE_error(y_pred,y_true):
    return np.mean(np.mean(np.abs(y_pred-y_true)/np.abs(y_true)) )
def abs_MAPE_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)/np.abs(y_true)
def abs_error(y_pred,y_true):
    return np.mean(np.mean(np.abs(y_pred-y_true)))  
def abs_error_element(y_pred,y_true):
    return np.abs(y_pred-y_true)

In [22]:
error_list = []
Ytrain_all = pd.DataFrame()
XGBR = xgb.XGBRegressor(max_depth = 3,learning_rate=0.1,n_estimators=500)
TRAIN_TST_C = ['SB00', 'SB01', 'SB02', 'SB03', 'SB04', 'SB05']
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values,Y.values[:,ind] )
    y_true = Y.values[:,ind]
    y_pred = XGBR.predict(X.values)   
    time2 = time.time()  
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))
    
    Ytrain =  XGBR.predict(X.values)
    Ytrain_df = pd.DataFrame(Ytrain)
    Ytrain_all = pd.concat((Ytrain_all,Ytrain_df),axis = 1)   
print(np.mean(error_list))

0_error:0.175139143475__time:1.8974356651306152
1_error:0.17712663309__time:1.4909651279449463
2_error:0.182400769645__time:1.4839460849761963
3_error:0.184267235304__time:1.535081386566162
4_error:0.185550072803__time:1.494473934173584
5_error:0.187195219751__time:1.5009913444519043
0.181946512345


In [23]:
#%%
Ytrain_true = pd.DataFrame(Y)
Y_error = abs_MAPE_error_element(Ytrain_all.values,Ytrain_true.values  )  
Good_ind =  Y_error.sum(axis=1).argsort()[0:np.int(0.90*len(X))]
#%%

In [24]:
XGBR = xgb.XGBRegressor(max_depth = 5,learning_rate=0.01,n_estimators=1600,reg_alpha=1,reg_lambda=0)
Ytest_all = pd.DataFrame()
error_list = []
Ytrain_all = pd.DataFrame()
XGBR = xgb.XGBRegressor(max_depth = 3,learning_rate=0.1,n_estimators=500)
TRAIN_TST_C = ['SB00', 'SB01', 'SB02', 'SB03', 'SB04', 'SB05']
for ind in range(6):
    time1 = time.time()
    XGBR.fit(X.values[Good_ind],Y.values[Good_ind,ind] )
    y_true = Y.values[Good_ind,ind]
    y_pred = XGBR.predict(X.values[Good_ind])
    time2 = time.time()    
    print(str(ind)+ '_error:' + str(abs_MAPE_error(y_pred,y_true  ) ) + '__time:'+ str(time2 - time1) )
    error_list.append(abs_MAPE_error(y_pred, y_true  ))
    Ytest = XGBR.predict(Xtest.values)
    Ytest_df = pd.DataFrame(Ytest)
    Ytest_all = pd.concat((Ytest_all,Ytest_df),axis = 1)    
print(np.mean(error_list))

0_error:0.15489646396__time:1.376659870147705
1_error:0.156754588966__time:1.376159429550171
2_error:0.161972565892__time:1.3786659240722656
3_error:0.161128978951__time:1.389695644378662
4_error:0.161615357866__time:1.3826770782470703
5_error:0.163408199383__time:1.3997223377227783
0.159962692503


In [33]:
# Gernerate Final Result

In [26]:
Xtest = pd.read_csv('C:/Users/Administrator/PycharmProjects/Traffic/Feature/Xtest.csv')
Ytest_all.columns = [0,1,2,3,4,5]
result = pd.concat([Ytest_all,Xtest],axis=1)
result = result.loc[:,[0,1,2,3,4,5,'TOLLGATE_ID','INTERSECTION_ID','TRN_STA','TRN_END']]
result['TRN_END'] = result['TRN_STA'].apply(lambda x :
str((datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours=2))))

In [32]:
result

,0,1,2,3,4,5,TOLLGATE_ID,INTERSECTION_ID,TRN_STA,TRN_END
0,48.537807,56.696766,46.691444,41.648350,52.086933,97.363716,2,A,2016-10-18 06:00:00,2016-10-18 08:00:00
1,144.625168,156.071350,125.277512,114.692413,87.683846,110.904907,3,A,2016-10-18 06:00:00,2016-10-18 08:00:00
2,100.522850,116.314926,90.910072,103.570656,90.437454,98.187386,1,B,2016-10-18 06:00:00,2016-10-18 08:00:00
3,62.027416,79.246033,63.670860,49.716572,43.049603,52.455299,3,B,2016-10-18 06:00:00,2016-10-18 08:00:00
4,193.778107,193.728989,169.027527,170.970535,156.303604,162.611023,1,C,2016-10-18 06:00:00,2016-10-18 08:00:00
5,151.174515,145.487976,130.752594,125.473145,114.683327,107.610222,3,C,2016-10-18 06:00:00,2016-10-18 08:00:00
6,70.967072,51.973141,53.885471,49.326653,48.964954,56.009094,2,A,2016-10-18 15:00:00,2016-10-18 17:00:00
7,120.161728,97.296524,96.141167,76.503197,81.548721,77.907043,3,A,2016-10-18 15:00:00,2016-10-18 17:00:00
8,121.117088,93.742996,102.214188,86.857635,80.027107,79.319687,1,B,2016-10-18 15:00:00,2016-10-18 17:00:00
9,103.582771,44.868225,57.123936,33.934139,37.268772,54.345139,3,B,2016-10-18 15:00:00,2016-10-18 17:00:00


In [34]:
sub_result = pd.DataFrame()
for x in range(result.shape[0]):
    t = result.loc[x,:]
    for y in range(6):
        end_time = str(datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*(y+1)))
        start_time = str((datetime.datetime.strptime(t['TRN_END'],'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=20*y)))
        timewindow = '['+start_time+','+end_time+')'
        each_result = pd.DataFrame({'intersection_id':[t['INTERSECTION_ID']],'tollgate_id':[t['TOLLGATE_ID']]
        ,'time_window':[timewindow] ,'avg_travel_time':[t[y]]})  
        sub_result = pd.concat((sub_result,each_result),axis=0)

In [35]:
sub_result

,avg_travel_time,intersection_id,time_window,tollgate_id
0,48.537807,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",2
0,56.696766,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",2
0,46.691444,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",2
0,41.648350,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",2
0,52.086933,A,"[2016-10-18 09:20:00,2016-10-18 09:40:00)",2
0,97.363716,A,"[2016-10-18 09:40:00,2016-10-18 10:00:00)",2
0,144.625168,A,"[2016-10-18 08:00:00,2016-10-18 08:20:00)",3
0,156.071350,A,"[2016-10-18 08:20:00,2016-10-18 08:40:00)",3
0,125.277512,A,"[2016-10-18 08:40:00,2016-10-18 09:00:00)",3
0,114.692413,A,"[2016-10-18 09:00:00,2016-10-18 09:20:00)",3


In [36]:
cols = ['intersection_id','tollgate_id','time_window','avg_travel_time']
sub_result = sub_result.ix[:, cols]

In [37]:
sub_result.to_csv('XGBOOST1.csv',index=False,)